In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%pip install wandb
!wandb

In [ ]:
%pip install kaggle --upgrade
import os
fpath = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")
if not os.path.isfile(fpath):
    try:
        os.mkdir(os.path.join(os.path.expanduser("~"), ".kaggle"))
    except Exception:
        pass
    with open(fpath, "w") as f:
        f.write(input(f"input {fpath}"))
    !chmod 600 {fpath}

In [ ]:
if 'datasets' not in os.listdir(os.getcwd()) or 'animals10' not in os.listdir(os.path.join(os.getcwd(), 'datasets')):
    !kaggle datasets download alessiocorrado99/animals10
    !mkdir datasets
    !unzip animals10.zip -d datasets/animals10
    !rm animals10.zip

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class someNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(IM_HEIGHT*IM_WIDTH*3, 100)
        self.l2 = nn.Linear(100, 10)
    def forward(self, x):
        x = self.l1(x)
        x = F.relu(x)
        x = self.l2(x)
        x = F.log_softmax(x)
        return x


In [ ]:
from tqdm import tqdm
def train(model, dev, train_set_loader, optimizer, epoch, WAND, LOG_INTERVAL):
    model.train()
    for bI, (x_b, y_b) in enumerate(tqdm(train_set_loader)):
        x_b = review(x_b)
        x_b, y_b = preprocess(x_b, y_b)
        pred = model.forward(x_b)
        loss = loss_fn(pred, y_b)
        # clear gradients for this training step
        optimizer.zero_grad()
        # backprop, compute gradients
        loss.backward()
        # apply gradients
        optimizer.step()
        if bI % LOG_INTERVAL == 0:
            loss_final = loss.mean().item()
            if WAND: wandb.log({"train_loss": loss_final})
            print(epoch, loss_final)

def test(model, dev, test_set_loader, WAND):
    with torch.no_grad():
        val_loss = torch.Tensor(len(test_set_loader)).to(dev)
        for i, (x_v_b, y_v_b) in enumerate(tqdm(test_set_loader)):
            x_v_b, y_v_b = preprocess(x_v_b, y_v_b)
            x_v_b = review(x_v_b)
            pred = model.forward(x_v_b)
            loss = loss_fn(pred, y_v_b)
            val_loss[i] = (loss.sum()/y_v_b.shape[0])
        val_loss_final = val_loss.mean().item()
        if WAND: wandb.log({"val_loss": val_loss_final})
        print(val_loss_final)


In [ ]:
WAND = True
LOG_INTERVAL = 1
N_EPOCH = 1
LR = 0.01
BS = 256
IM_HEIGHT = 425
IM_WIDTH = 600
TRAIN_DAT_PERC = 0.95

In [ ]:
if WAND:
  import wandb
  wandb_run = wandb.init(project="animals10", entity="0xasim")
  wandb.config = {
    "learning_rate": LR,
    "epochs": N_EPOCH,
    "batch_size": BS
  }
  run_name = wandb_run.name

In [ ]:
import torch
import os
torch.manual_seed(17)
from torchvision import datasets, transforms
transform = transforms.Compose([transforms.Resize([IM_HEIGHT, IM_WIDTH]), transforms.ToTensor()])
path = os.path.join(os.getcwd(), "datasets", "animals10", "raw-img")
dataset = datasets.ImageFolder(path, transform=transform)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*TRAIN_DAT_PERC), int(len(dataset)*round(1-TRAIN_DAT_PERC, 2))+1])
train_set_loader = torch.utils.data.DataLoader(train_set, batch_size=BS, shuffle=True)
test_set_loader = torch.utils.data.DataLoader(test_set, batch_size=BS, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
i, l = next(iter(train_set_loader))
print(i.shape, i[0].shape)
sample = i[0][2]
print(sample)
plt.imshow(i[0][2].reshape(IM_HEIGHT, IM_WIDTH))
plt.show()
print(i[0][0])
print(torch.min(i[0][0]))
print(torch.max(i[0][0]))

In [ ]:
def preprocess(x, y):
    return x.to(dev), y.to(dev)
review = lambda x_b: x_b.view(len(x_b), -1)

if __name__ == '__main__':
    dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(dev)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    model = someNet()
    model.to(dev)
    loss_fn = nn.CrossEntropyLoss().to(dev)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    for epoch in range(N_EPOCH):
        train(model, dev, train_set_loader, optimizer, epoch, WAND, LOG_INTERVAL)
        test(model, dev, test_set_loader, WAND)

In [ ]:
model_name = run_name+'.pt'
MODEL_PATH = os.path.join(os.getcwd(), str(N_EPOCH))
torch.save(model.state_dict(), model_name)
print(model_name)
!ls -lah {model_name}

In [ ]:
INCOLAB = 'google.colab' in str(get_ipython()) if hasattr(__builtins__,'__IPYTHON__') else False
DOWNLOAD = True
if INCOLAB and DOWNLOAD:
    from google.colab import files
    files.download(model_name)